### Student Information
Name: 謝尚哲

Student ID: 114064545

GitHub ID: shangchehsieh

---

### Instructions

### First Phase Submission

1. First: do the **take home** exercises in the [DM2025-Lab1-Master](https://github.com/leoson-wu/DM2025-Lab1-Exercise/blob/main/DM2025-Lab1-Master.ipynb) that considered as **phase 1 (from exercise 1 to exercise 15)**. You can answer in the master file. __This part is worth 10% of your grade.__


2. Second: follow the same process from the [DM2025-Lab1-Master](https://github.com/leoson-wu/DM2025-Lab1-Exercise/blob/main/DM2025-Lab1-Master.ipynb) on **the new dataset** up **until phase 1**. You can skip some exercises if you think some steps are not necessary. However main exercises should be completed. You don't need to explain all details as we did (some **minimal comments** explaining your code are useful though).  __This part is worth 15% of your grade.__
    -  Use [the new dataset](https://github.com/leoson-wu/DM2025-Lab1-Exercise/blob/main/newdataset/Reddit-stock-sentiment.csv). The dataset contains a 16 columns including 'text' and 'label', with the sentiment labels being: 1.0 is positive, 0.0 is neutral and -1.0 is negative. You can simplify the dataset and use only the columns that you think are necessary. 
    
    - You are allowed to use and modify the `helper` functions in the folder of the first lab session (notice they may need modification) or create your own.
    - Use this file to complete the homework from the second part. Make sure the code can be run from the beginning till the end and has all the needed output.


3. Third: please attempt the following tasks on **the new dataset**. __This part is worth 10% of your grade.__
    - Generate meaningful **new data visualizations**. Refer to online resources and the Data Mining textbook for inspiration and ideas. 
    


4. Fourth: It's hard for us to follow if your code is messy, so please **tidy up your notebook** and **add minimal comments where needed**. __This part is worth 5% of your grade.__

You can submit your homework following these guidelines: [DM2025-Lab1-announcement](https://github.com/leoson-wu/DM2025-Lab1-Announcement/blob/main/README.md). Make sure to commit and save your changes to your repository __BEFORE the deadline (September 28th 11:59 pm, Sunday)__. 

### Second Phase Submission 

**You can keep the answer for phase 1 for easier running and update the phase 2 on the same page.**

1. First: Continue doing the **take home** exercises in the [DM2025-Lab1-Master](https://github.com/leoson-wu/DM2025-Lab1-Exercise/blob/main/DM2025-Lab1-Master.ipynb) for **phase 2, starting from Finding frequent patterns**. Use the same master(.ipynb) file. Answer from phase 1 will not be considered at this stage. You can answer in the master file. __This part is worth 10% of your grade.__


2. Second: Continue from first phase and do the same process from the [DM2025-Lab1-Master](https://github.com/leoson-wu/DM2025-Lab1-Exercise/blob/main/DM2025-Lab1-Master.ipynb) on **the new dataset** for phase 2, starting from Finding frequent pattern. You can skip some exercises if you think some steps are not necessary. However main exercises should be completed. You don't need to explain all details as we did (some **minimal comments** explaining your code are useful though).  __This part is worth 15% of your grade.__
    - Continue using this file to complete the homework from the second part. Make sure the code can be run from the beginning till the end and has all the needed output. Use the same new dataset as in phase 1.
    
    - You are allowed to use and modify the `helper` functions in the folder of the first lab session (notice they may need modification) or create your own.

3. Third: please attempt the following tasks on **the new dataset**. __This part is worth 20% of your grade.__
    - Use this file to answer.
    - Generate **TF-IDF features** from the tokens of each text. This will generating a document matrix, however, the weights will be computed differently (using the TF-IDF value of each word per document as opposed to the word frequency).  Refer to this Scikit-learn [guide](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) .
    - Implement a simple **Naive Bayes classifier** that automatically classifies the records into their categories. Use both the TF-IDF features and word frequency features to build two seperate classifiers. Note that for the TF-IDF features you might need to use other type of NB classifier different than the one in the Master Notebook. Comment on the differences and when using augmentation with feature pattern.  Refer to this [article](https://hub.packtpub.com/implementing-3-naive-bayes-classifiers-in-scikit-learn/).


4. Fourth: In the lab, we applied each step really quickly just to illustrate how to work with your dataset. There are somethings that are not ideal or the most efficient/meaningful. Each dataset can be handled differently as well. What are those inefficent parts you noticed? How can you improve the Data preprocessing for these specific datasets? __This part is worth 10% of your grade.__


5. Fifth: It's hard for us to follow if your code is messy, so please **tidy up your notebook** and **add minimal comments where needed**. __This part is worth 5% of your grade.__


You can submit your homework following these guidelines: [DM2025-Lab1-announcement](https://github.com/leoson-wu/DM2025-Lab1-Announcement/blob/main/README.md). Make sure to commit and save your changes to your repository __BEFORE the deadline (October 19th 11:59 pm, Sunday)__. 

# Phase 1

In [1]:
### Begin Assignment Here

### 1. Data Preparation

In [51]:
# test code for environment setup
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt') # download the NLTK datasets
from sklearn.utils import Bunch
from sklearn.feature_extraction.text import CountVectorizer
import plotly as py
import math
import PAMI
import umap
df = pd.read_csv('newdataset/Reddit-stock-sentiment.csv')

needed_cols = ["text", "subjectivity", "polarity", "sentiment", "label"]
df = df[needed_cols].copy()

# 基本型別與值域檢查
df["text"] = df["text"].astype(str)
df["subjectivity"] = pd.to_numeric(df["subjectivity"], errors="coerce")
df["polarity"] = pd.to_numeric(df["polarity"], errors="coerce")
df["label"] = pd.to_numeric(df["label"], errors="coerce")  # ground truth: {-1.0, 0.0, 1.0}

# 保留合法標籤
df = df[df["label"].isin([-1.0, 0.0, 1.0])].dropna(subset=["text"]).reset_index(drop=True)

#建立 label 對應（-1,0,1 → 0,1,2）
label_order = [-1.0, 0.0, 1.0]
categories = ["negative", "neutral", "positive"]
label_to_idx = {lab: i for i, lab in enumerate(label_order)}

#  準備 sklearn 友善的結構 (Bunch)
X_text = df["text"].tolist()
y = df["label"].map(label_to_idx).to_numpy()

twenty_train = Bunch(
    data=X_text,             # list[str]
    target=y,                # np.ndarray of ints {0,1,2}
    target_names=categories  # ['negative','neutral','positive']
)

# 同時輸出一個乾淨的分析表 (X DataFrame) ----
#   方便你在 Notebook 做 EDA、畫圖或自訂特徵工程
X = df[["text", "subjectivity", "polarity","sentiment", "label"]].copy()

# （選配）如果你想同時保留數字化標籤以便 join / 分層抽樣：
X["label_idx"] = X["label"].map(label_to_idx)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aska9\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 2. Data Transformation

#### 2.1 Converting Dictionary into Pandas Dataframe

In [52]:
# my functions
import helpers.data_mining_helpers as dmh

# construct dataframe from a list
X = pd.DataFrame.from_records(dmh.format_rows(twenty_train), columns= ['text'])
X

,text
0,Calls on retards
1,Stunt as in like why did they even make a big ...
2,Seeing lots of red in the ticker.
3,Vision Marine Technologies Inc. is rewriting t...
4,He didn’t say thank you.
...,...
842,Invest now it is going to shoot up tomorrow
843,Where can I read this?
844,"Upgrades, Downgrades\r \r [Stock Analyst Reco..."
845,i sold my puts so i expect cheeto to declare w...


#### 2.2 Adding Columns

- Get the key value via `target`

In [53]:
X['sentiment'] = twenty_train.target
X

,text,sentiment
0,Calls on retards,0
1,Stunt as in like why did they even make a big ...,1
2,Seeing lots of red in the ticker.,1
3,Vision Marine Technologies Inc. is rewriting t...,2
4,He didn’t say thank you.,0
...,...,...
842,Invest now it is going to shoot up tomorrow,2
843,Where can I read this?,1
844,"Upgrades, Downgrades\r \r [Stock Analyst Reco...",1
845,i sold my puts so i expect cheeto to declare w...,0


- Get sentiment name from the key of sentiment

In [54]:
X['sentiment_name'] = X.sentiment.apply(lambda t: dmh.format_labels(t, twenty_train))
X

,text,sentiment,sentiment_name
0,Calls on retards,0,negative
1,Stunt as in like why did they even make a big ...,1,neutral
2,Seeing lots of red in the ticker.,1,neutral
3,Vision Marine Technologies Inc. is rewriting t...,2,positive
4,He didn’t say thank you.,0,negative
...,...,...,...
842,Invest now it is going to shoot up tomorrow,2,positive
843,Where can I read this?,1,neutral
844,"Upgrades, Downgrades\r \r [Stock Analyst Reco...",1,neutral
845,i sold my puts so i expect cheeto to declare w...,0,negative


#### 2.3 Familiarizing yourself with the Data - fetch records belonging to "posivive" and query every 10th record. Only show the first 5 records.

In [58]:
X[X["sentiment_name"] == "positive"][::10][0:5]

,text,sentiment,sentiment_name
3,Vision Marine Technologies Inc. is rewriting t...,2,positive
78,I think it’s gonna work itself out. I bought t...,2,positive
168,The obvious can also be refreshing from CNBC: ...,2,positive
219,Same. Back to break even after that last rip b...,2,positive
319,Etrade has a feed of news articles about how r...,2,positive


### 3. Data Mining using Pandas

#### 3.1 Missing Values

- `.isnull()`: tranformed our table into the True/False table
- `.check_missing_values()`: looks through each field (attribute or column) in the dataset and counts how many missing values were found.

In [59]:
X.isnull().apply(lambda x: dmh.check_missing_values(x))

,text,sentiment,sentiment_name
0,The amoung of missing records is:,The amoung of missing records is:,The amoung of missing records is:
1,0,0,0


I try to calculate the missing values in every record instead of every column.

In [60]:
X.isnull().apply(lambda x: dmh.check_missing_values(x), axis=1)

0      (The amoung of missing records is: , 0)
1      (The amoung of missing records is: , 0)
2      (The amoung of missing records is: , 0)
3      (The amoung of missing records is: , 0)
4      (The amoung of missing records is: , 0)
                        ...                   
842    (The amoung of missing records is: , 0)
843    (The amoung of missing records is: , 0)
844    (The amoung of missing records is: , 0)
845    (The amoung of missing records is: , 0)
846    (The amoung of missing records is: , 0)
Length: 847, dtype: object

# Phase 2

In [ ]:
### Begin Assignment Here